# Métodos "clásicos" de aprendizaje estadístico

## Este notebook contiene 3 ejercicios:
1. Hacer nuestra propia implementación de KNN-I.
2. Contruir un pipeline para aplicar KNN-I usando scikit-learn.
3. Implementar MissForest usando los bloques de construcción que ofrece scikit-learn.
4. Elegir un conjunto de metricas y comparar las 2 tecnicas !

Primero importemos todas las dependencias necesarias

In [1]:
import time
import copy
import math
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

Luego, importemos los datos meteorológicos

In [2]:
marengo_df = pd.read_csv("../src/marengo_nans.csv")

## KNN-I

Gran parte de los algoritmos de aprendizaje automático estan implementados en elegantes bibliotecas que permiten su rapida ejecución bajo el tradicional formato de dos métodos: ```.fit()``` y ```.predict()```.Esto reduce los algoritmos a una caja negra que algunos usuarios utilizan indiscriminadamente. La falta de comprensión del funcionamiento de los métodos es un riesgo para la calidad de los resultados.

Aprovechando que KNN es un algoritmo de facil implementación, vamos a escribirlo nosotros mismos para interiorizar el funcionamiento. En un primer momento vamos a hacerlo con python puro.

### Ex.1: Hacer nuestra propia implementación de KNN-I.

Como lo mencionamos durante la presentación el algoritmo de los KNN consta de tres étapas.

#### 1.Cálculo de la distancia entre observaciones.
**Nota:** Nuestra implementación sera secuencial y busca ser lo más simple posible, para hacerla mas comprensible. Por lo tanto sacrificaremos la complejidad computacional de nuestro algoritmo.

In [3]:
def eucledian_distance_nans(a, b):
    """
    Una función que calcule la distancia euclidiana entre dos vectores.
    La distancia euclidiana entre dos puntos en un espacio n-dimensional es la raíz cuadrada de
    la suma de las diferencias al cuadrado entre las coordenadas correspondientes de los puntos.
    Esta implementación debe ser robusta a los valores faltantes dentro de los vectores.
    """
    # Paso 1: Crear una mascara que indique los elementos que son NaN en uno o ambos vectores.
    mascara = []
    for i,j in zip(a, b):
        if i is None or j is None:
            mascara.append(False)
        else:
            mascara.append(True)

    # TODO Paso 2: Estimar el ponderador contando los elementos que son Nan: # elemetos/# elementos comparables.
    weight = len(mascara)/mascara.count(True)

    # Paso 3: Calcular la distancia eucladiana considerando solo los elementos no nulos. Devolver dicha distancia.
    sumatoria = 0
    for idx in range(len(a)):
        # TODO Calcular la sumatoria al cuadrado interna de la distancia eucladiana incluyendo solo los elementos no nulos
        if mascara[idx]:
            sumatoria += (a[idx]-b[idx])**2

    return round(math.sqrt(weight * sumatoria), 8)

In [4]:
vec_1 = [0., 10., 5., None, 5., 1., 5., None]
vec_2 = [1., None, 0., 7., 0., 2., 3., 10.]
vec_3 = [15., 120., 10., 1., 2., 7., 15., 8.]
print(eucledian_distance_nans(vec_1, vec_2))
print(eucledian_distance_nans(vec_2, vec_3))
print(eucledian_distance_nans(vec_3, vec_1))

# LA RESPUESTA DEBE SER:
# 9.46572765
# 24.11875382
# 129.0736224

9.46572765
24.11875382
129.0736224


In [5]:
def calculate_distance_matrix(X):
    ref_len = len(X[0])
    for vec in X:
        if len(vec) != ref_len:
            raise KeyError("No todos los vectores tienen la misma dimension")
    #TODO Paso 1: Crear una matriz de 0s con las dimensiones de la matriz de distancias. nxn donde n es el num. de vectores.
    matrix = [[0. for _ in range(len(X))] for _ in range(len(X))]
    #TODO: Llenar la matriz usando la función que acabamos de crear.
    for i in range(len(X)):
        for j in range(len(X)):
            matrix[i][j] = eucledian_distance_nans(X[i],X[j])
    return matrix
    

In [6]:
vec_4 = [0., None, 10., 17., 20., 32., 5., 0.]
vec_5 = [15., 120., 10., None, 3., 1., 0., 0.]
matrix_1 = [vec_1, vec_2, vec_3, vec_4, vec_5]

print("Matriz:")
for i in matrix_1:
    print(i)
print("\n")

dist_matrix = calculate_distance_matrix(matrix_1)
print("Matriz de distancias:")
for i in dist_matrix:
    print(i)

# LA RESPUESTA DEBE SER:
# Matriz de distancias:
# [0.0, 9.46572765, 129.0736224, 44.01817806, 128.47308408]
# [9.46572765, 0.0, 24.11875382, 42.8285619, 23.52303835]
# [129.0736224, 24.11875382, 0.0, 42.68154502, 19.30210943]
# [44.01817806, 42.8285619, 42.68154502, 0.0, 44.72135955]
# [128.47308408, 23.52303835, 19.30210943, 44.72135955, 0.0]

Matriz:
[0.0, 10.0, 5.0, None, 5.0, 1.0, 5.0, None]
[1.0, None, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
[15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
[0.0, None, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
[15.0, 120.0, 10.0, None, 3.0, 1.0, 0.0, 0.0]


Matriz de distancias:
[0.0, 9.46572765, 129.0736224, 44.01817806, 128.47308408]
[9.46572765, 0.0, 24.11875382, 42.8285619, 23.52303835]
[129.0736224, 24.11875382, 0.0, 42.68154502, 19.30210943]
[44.01817806, 42.8285619, 42.68154502, 0.0, 44.72135955]
[128.47308408, 23.52303835, 19.30210943, 44.72135955, 0.0]


#### 2. Cálculo de los k vecinos más cercanos

In [7]:
def generar_matriz_vecinos(k, X, X_dist):
    """
    Genera una matriz que guarda el indice de los k vecinos más cercanos de cada vector en la matriz de distancias.
    """
    # TODO Paso 1: Crear una matriz de 0s con las dimensiones de la matriz de vecinos. nxk donde n es el num. de vectores.
    matriz_vecinos = [[0 for _ in range(k)] for _ in range(len(X))]
    
    # Paso 2: Llenar la matriz usando la matriz de distancias.
    for i in range(len(X)):
        # TODO :Obtener los indices de los k vecinos más cercanos a X[i] ordenados por distancia. Recuerde eliminar el indice del propio vector !
        indices_vecinos = sorted(list(range(len(X_dist[i]))), key=lambda x: X_dist[i][x])
        indices_vecinos.remove(i)
        indices_vecinos = indices_vecinos[:k]

        matriz_vecinos[i] = indices_vecinos
    
    return matriz_vecinos
 

In [8]:
print(generar_matriz_vecinos(2, matrix_1, dist_matrix))
# LA RESPUESTA DEBE SER:
# [[1, 3], [0, 4], [4, 1], [2, 1], [2, 1]]

[[1, 3], [0, 4], [4, 1], [2, 1], [2, 1]]


#### 3. Imputar los valores faltantes usando la media de los k vecinos más cercanos

In [9]:
def knn_eucledian_imputer(X, k):
    """
    Imputa los valores faltantes en la matriz X utilizando el algoritmo KNN.
    """
    # Paso 1: Calcular la matriz de distancias entre los vectores de X.
    dist_matrix = calculate_distance_matrix(X)
    # Paso 2: Generar la matriz de indices de los k vecinos más cercanos.
    vecinos = generar_matriz_vecinos(k, X, dist_matrix)
    # Paso 3: Crear una copia de X para almacenar los valores imputados.
    X_imputed = copy.deepcopy(X)

    # Paso 4: Iterar sobre cada vector en X.
    for idx in range(len(X)):
        # Paso 5: Iterar sobre cada elemento del vector.
        for jdx in range(len(X[idx])):
            # Si el elemento es NaN, imputar su valor.
            if X[idx][jdx] is None:
                # TODO Paso 6: Obtener los valores de los k vecinos más cercanos.
                indices_vecinos = vecinos[idx]
                valores_vecinos_nonan = [X[vecino][jdx] for vecino in indices_vecinos if X[vecino][jdx] is not None]
                # TODO Paso 7: Calcular la media de los valores de los vecinos y asignarla al elemento faltante. Ojo, considerar la excepción de que no haya vecinos con valores no NaN.
                if valores_vecinos_nonan:
                    X_imputed[idx][jdx] = sum(valores_vecinos_nonan) / len(valores_vecinos_nonan)
                else:
                    X_imputed[idx][jdx] = None
    
    return X_imputed

In [10]:
imputed_matrix = knn_eucledian_imputer(matrix_1, 2)
print("Matriz imputada:")
for i in imputed_matrix:
    print(i)

# LA RESPUESTA DEBE SER:
# [0.0, 10.0, 5.0, 12.0, 5.0, 1.0, 5.0, 5.0]
# [1.0, 65.0, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
# [15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
# [0.0, 92.5, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
# [15.0, 120.0, 10.0, 4.0, 3.0, 1.0, 0.0, 0.0]

Matriz imputada:
[0.0, 10.0, 5.0, 12.0, 5.0, 1.0, 5.0, 5.0]
[1.0, 65.0, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
[15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
[0.0, 120.0, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
[15.0, 120.0, 10.0, 4.0, 3.0, 1.0, 0.0, 0.0]


#### **Probemos nuestra implementación en los datos reales!**

In [11]:
marengo_lists = marengo_df.iloc[:,4:].values.tolist()
marengo_lists = [[None if pd.isna(x) else x for x in row] for row in marengo_lists]
start_time = time.time()
imputed_marengo = knn_eucledian_imputer(marengo_lists, 10)
end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")
print("Matriz imputada de Marengo:")
for i in imputed_marengo[:10]:  # Imprimimos solo las primeras 10 filas por brevedad
    print(i)

Tiempo de ejecución: 9.00 segundos
Matriz imputada de Marengo:
[13.508667, 19.367, 8.467, 1.9333334599999996, 42680.883, 13.708391, 35.28, 20.6, 3.4468565]
[14.596166, 19.317001, 11.167, 5.2999997, 42684.59, 13.493999, 33.48, 21.49, 3.663233]
[13.248249, 20.417, 8.217, 3.420000125, 42688.617, 10.446206, 30.599998, 23.35, 3.798374866666666]
[12.804502, 19.417, 7.467, 1.3000001, 42692.95, 14.05845, 37.44, 25.6, 4.251072]
[12.331582, 18.767, 5.267, 0.0, 42697.59, 15.39184, 36.36, 23.75, 3.9684093]
[12.266999, 18.517, 5.217, 0.0, 42702.523, 15.119999, 38.519997, 24.43, 3.959472]
[12.875333, 20.1795005, 5.817, 0.1, 43176.34922222223, 7.968939, 23.039999, 25.17, 4.401479]
[11.935749, 20.267, 4.467, 0.0, 43361.967, 15.876775, 39.239998, 26.24, 4.566951]
[13.164916, 20.522000400000003, 7.1670003, 0.0, 42719.04, 19.164717, 46.44, 23.83, 4.1956615]
[12.521167, 18.667, 6.517, 0.075, 42725.094, 23.277834, 53.639996, 26.48, 4.431484]


### Ex.2: Contruir un pipeline para aplicar KNN-I usando scikit-learn.

In [ ]:
# TODO Escalar los datos para mejorar la precisión del KNN usando la clase StandardScaler de sklearn.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(marengo_df.iloc[:, 4:])
# TODO usar la implementacion del algoritmo KNN de sklearn para imputar los valores faltantes de la matriz de Marengo.
# Este es el ejercicio más sencillo, ya que sklearn se encarga de todo el proceso de cálculo de distancias y vecinos.
# Puede resolverse usando solo dos lineas de codigo, una para crear una instacia de la clase KNNImputer y otra para imputar los valores faltantes.
imputer = KNNImputer(n_neighbors=2)
start_time = time.time()
imputed_array = imputer.fit_transform(marengo_df.iloc[:, 4:])
end_time = time.time()
# Invertir el escalado para que los datos imputados estén en la misma escala que los originales.
imputed_array = scaler.inverse_transform(imputed_array)
# Tomar los resultados y convertirlos de nuevo en DataFrame :)
imputed_df_knn = pd.DataFrame(imputed_array, columns=marengo_df.iloc[:, 4:].columns)
print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")
imputed_df_knn.head(10)



,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,25.043098,47.006362,21.231191,12.244961,2.888352e+07,75.931982,347.804676,86.987206,5.673011
1,25.977151,46.935064,25.301152,30.981112,2.888602e+07,74.963799,331.873203,89.876292,5.804465
2,24.819425,48.503652,20.854342,12.765409,2.888874e+07,61.200125,306.382829,95.914158,6.082544
3,24.438291,47.077662,19.723798,10.163167,2.889167e+07,77.512830,366.922443,103.218029,6.161591
4,24.032100,46.150768,16.407533,3.397333,2.889481e+07,83.534350,357.363559,97.212624,5.989867
5,23.976630,45.794270,16.332163,3.397333,2.889814e+07,82.306730,376.481300,99.420016,5.984437
6,24.499128,48.040206,17.236599,3.917782,2.923310e+07,50.012916,239.470652,101.822178,6.252967
7,23.692120,48.289753,15.201618,3.397333,2.928567e+07,85.724291,382.853898,105.295574,6.353495
8,24.747851,47.968906,19.271580,3.397333,2.890930e+07,100.572466,446.579807,97.472317,6.127928
9,24.194935,46.008169,18.291774,3.397333,2.891339e+07,119.147088,510.305662,106.074653,6.271195


### Ex.3: Implementar MissForest usando los bloques de construcción que ofrece scikit-learn.

In [ ]:
# TODO Escalar los datos
scaler = StandardScaler()
scaled_data = scaler.fit_transform(marengo_df.iloc[:, 4:])

# TODO Imputar usando IterativeImputer con RandomForestRegressor
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), 
                           max_iter=100, random_state=42)
start_time = time.time()
imputed_scaled = imputer.fit_transform(scaled_data)
end_time = time.time()

imputed_data = scaler.inverse_transform(imputed_scaled)
imputed_df_mf = pd.DataFrame(imputed_data, columns=marengo_df.iloc[:, 4:].columns)
print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")
imputed_df_mf.head(10)

/home/julagu/miniconda3/envs/ML_imputation/lib/python3.10/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,13.508667,19.367000,8.467,4.18,42680.8830,13.708391,35.280000,20.60,3.446856
1,14.596166,19.317001,11.167,5.30,42684.5900,13.493999,33.480000,21.49,3.663233
2,13.248249,20.417000,8.217,1.59,42688.6170,10.446206,30.599998,23.35,4.145288
3,12.804502,19.417000,7.467,1.30,42692.9500,14.058450,37.440000,25.60,4.251072
4,12.331582,18.767000,5.267,0.00,42697.5900,15.391840,36.360000,23.75,3.968409
5,12.266999,18.517000,5.217,0.00,42702.5230,15.119999,38.519997,24.43,3.959472
6,12.875333,20.317001,5.817,0.10,43181.6338,7.968939,23.039999,25.17,4.401479
7,11.935749,20.267000,4.467,1.16,43730.5948,15.876775,39.239998,26.24,4.566951
8,13.164916,20.277001,7.167,0.00,42719.0400,19.164717,46.440000,23.83,4.195661
9,12.521167,18.667000,6.517,0.16,42725.0940,23.277834,53.639996,26.48,4.431484


### Ex4. Elegir un conjunto de metricas y comparar las 2 tecnicas !

In [ ]:
# TODO Ver la documentacion de Sklearn y utilizar metricas de regresión para evaluar la calidad de la imputación. Recuerden que tenemos los datos reales !!!
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
def eval(name, y, y_hat):
    eval = {
        "model": name,
        "rmse": root_mean_squared_error(y,y_hat),
        "mae": mean_absolute_error(y, y_hat),
        "r2": r2_score(y, y_hat),
    }
    return eval

real_marengo = pd.read_csv("../src/marengo.csv")
real_marengo = real_marengo.iloc[:, 4:].to_numpy()
values = marengo_df.iloc[:, 4:].to_numpy()
nans_coords = np.argwhere(np.isnan(values))
array_knn = imputed_df_knn.to_numpy()
array_mf = imputed_df_mf.to_numpy()

In [15]:
results = [eval("KNNI", real_marengo[nans_coords[:, 0], nans_coords[:, 1]],
                array_knn[nans_coords[:, 0], nans_coords[:, 1]]),
           eval("MissForest", real_marengo[nans_coords[:, 0], nans_coords[:, 1]],
                array_mf[nans_coords[:, 0], nans_coords[:, 1]])]

df_results = pd.DataFrame(results)

In [16]:
print(results)

[{'model': 'KNNI', 'rmse': 9989530.422944533, 'mae': 3382648.2023601444, 'r2': -516055.06926382356}, {'model': 'MissForest', 'rmse': 208.3586842506935, 'mae': 57.31706602384783, 'r2': 0.9997754929646829}]


In [17]:
df_results

,model,rmse,mae,r2
0,KNNI,9.989530e+06,3.382648e+06,-516055.069264
1,MissForest,2.083587e+02,5.731707e+01,0.999775
